In [1]:
# default_exp optimizer
# all_slow

# Optimizer
> Utility functions for FastAI optimizers in Pytorch models.

Some utility functions for training Pytorch models with FastAI fine tuning method. The code is from [this repository](https://github.com/muellerzr/fastai_minima/blob/master/fastai_minima/optimizer.py#L159).

In [2]:
# export
import torch.nn as nn
from fastcore.foundation import L
from fastai.optimizer import OptimWrapper

In [3]:
# export
def params(m):
    "Return all parameters of `m` (Pytorch model)."
    return [p for p in m.parameters()]

In [4]:
# export
def convert_params(o:list) -> list:
    """
    Converts `o` into Pytorch-compatable param groups
    `o` should be a set of layer-groups that should be split in the optimizer
    Example:
    ```python
    def splitter(m): return convert_params([[m.a], [m.b]])
    ```
    Where `m` is a model defined as:
    ```python
    class RegModel(Module):
      def __init__(self): self.a,self.b = nn.Parameter(torch.randn(1)),nn.Parameter(torch.randn(1))
      def forward(self, x): return x*self.a + self.b
    ```
    """
    if not isinstance(o[0], dict):
        splitter = []
        for group in o:
            if not isinstance(group[0], nn.parameter.Parameter):
                group = L(group).map(params)[0]
            splitter.append({'params':group})
        return splitter
    return o

In the FastAI library is largely used transfer learning with layer-group learning rate freezing. 
The `convert_params` function returns a list of parameters for specific layers in a model that allows discriminative learning rates.

In [ ]:
#export
def smp_splitter(model):
    return convert_params([[model.encoder],[model.decoder],[model.segmentation_head]])

In [1]:
#export
def opt_func(params, torch_opt, *args, **kwargs):
    """Pytorch Optimizer for fastai Learner."""
    return OptimWrapper(params, torch_opt, *args, **kwargs)